In [1]:
import numpy as np

# ignore_box2_or_not

In [2]:
def find_inner_box_isin_outer_box(box1: list, box2: list, scale: int = 5) -> bool:
    """determine whether a box is in another box

    Args:
        outer_box (list): 假设外部盒子 [x_min, y_min, x_max, y_max]
        inner_box (list): 假设内部盒子 [x_min, y_min, x_max, y_max]
        scale (int):      调整盒子的大小,相当于忽略宽高输出scale的大小. Defaults to 5.

    Returns:
        bool: 外部盒子是否包含内部盒子
    """
    # 外面包裹内部
    left   = int(box1[0] / scale) - int(box2[0] / scale) # < 0 说明outer_box更靠左
    top    = int(box1[1] / scale) - int(box2[1] / scale) # < 0 说明outer_box更靠上
    right  = int(box1[2] / scale) - int(box2[2] / scale) # > 0 说明outer_box更靠右
    bottom = int(box1[3] / scale) - int(box2[3] / scale) # > 0 说明outer_box更靠下

    if left <= 0 and top <= 0 and right >= 0 and bottom >= 0:
        return True
    else:
        return False

In [3]:
# box1, box2
boxes = [
    [[50, 50, 100, 100], [60, 60, 90, 90]],
    [[50, 50, 100, 100], [110, 110, 120, 120]],
    [[50, 50, 100, 100], [10, 10, 20, 20]],
    [[50, 50, 100, 100], [95, 95, 105, 105]],
    [[50, 50, 100, 100], [95, 95, 102, 102]],
    [[50, 50, 100, 100], [45, 45, 55, 55]],
    [[50, 50, 100, 100], [48, 48, 55, 55]],
]

for box1, box2 in boxes:
    print(find_inner_box_isin_outer_box(box1, box2))

True
False
False
False
True
False
False


In [4]:
def ignore_box2_or_not(box1: list, box2: list, ratio: float = 0.75) -> bool:
    """determine whether a box is in another box

    Args:
        box1 (list): 假设外部盒子 [x_min, y_min, x_max, y_max]
        box2 (list): 假设内部盒子 [x_min, y_min, x_max, y_max]
        ratio (float): inner_box相当于box2的面积的阈值,大于阈值就忽略. Defaults to 0.75.

    Returns:
        bool: 外部盒子是否包含内部盒子
    """
    # 内部盒子面积
    print(max(box1[0], box2[0]))
    inner_box_x1 = max(box1[0], box2[0])
    inner_box_y1 = max(box1[1], box2[1])
    inner_box_x2 = min(box1[2], box2[2])
    inner_box_y2 = min(box1[3], box2[3])
    # max 用来判断是否重叠
    inner_box_area = max(inner_box_x2 - inner_box_x1, 0) * max(inner_box_y2 - inner_box_y1, 0)

    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    print(inner_box_area / box2_area)
    if inner_box_area / box2_area > ratio:
        return True
    else:
        return False

In [5]:
# box1, box2
boxes = [
    [[50, 50, 100, 100], [60, 60, 90, 90]],
    [[50, 50, 100, 100], [110, 110, 120, 120]],
    [[50, 50, 100, 100], [10, 10, 20, 20]],
    [[50, 50, 100, 100], [95, 95, 105, 105]],
    [[50, 50, 100, 100], [95, 95, 102, 102]],
    [[50, 50, 100, 100], [45, 45, 55, 55]],
    [[50, 50, 100, 100], [48, 48, 55, 55]],
]

for box1, box2 in boxes:
    print(ignore_box2_or_not(box1, box2))

60
1.0
True
110
0.0
False
50
0.0
False
95
0.25
False
95
0.5102040816326531
False
50
0.25
False
50
0.5102040816326531
False


# ignore_boxes

In [6]:
detections = np.array([[ 10, 0.8, 200.04971, 196.26697, 489.98325, 424.07892],
                        [10, 0.7, 141.04881, 311.3442 , 228.94856, 408.5379 ],
                        [10, 0.6, 0.       , 303.4387 , 175.52124, 424.90558],
                        [10, 0.5, 176.42613, 0.       , 460.68604, 227.06232],
                        [10, 0.3, 384.6766 , 283.063  , 419.97977, 335.35898],
                        [10, 0.8, 97.71875 , 346.97867, 103.96518, 353.037  ],
                        [10, 0.7, 575.25476, 195.62448, 628.17926, 291.2721 ],
                        [11, 0.6, 450.49182, 1.8310547, 640.     , 292.99066],
                        [11, 0.7, 73.79396 , 368.1626 , 79.10231 , 372.40448],
                        [10, 0.9, 84.013214, 332.34296, 89.18914 , 337.10605],
                        [12, 0.8, 596.2429 , 248.21837, 601.9428 , 253.99461],
                        [10, 0.1, 372.0439 , 363.4396 , 378.0838 , 368.31393]])

In [7]:
# 获取每个类别
classes = np.unique(detections[:, 0])
classes

array([10., 11., 12.])

In [23]:
new_detections = []
# 遍历单一类别
for cls in classes:
    dets_sig_cls = detections[detections[:, 0] == cls]
    # 如果一个类别只有1个框,就直接保存
    if len(dets_sig_cls) == 1:
        new_detections.append(dets_sig_cls)
        continue
    # 求面积,根据面积排序,不是最好的办法
    h = dets_sig_cls[:, 5] - dets_sig_cls[:, 3]
    w = dets_sig_cls[:, 4] - dets_sig_cls[:, 2]
    area = np.array(h * w)
    index = area.argsort()              # 得到面积排序index
    index = index[::-1]                 # 转换为降序
    dets_sig_cls = dets_sig_cls[index]  # dets_sig_cls转换为相同的顺序(这样做似乎可以解决index不匹配问题)

    # max代表大的框,min代表小的框
    keeps = []
    for i, max_i in enumerate(index[:-1]):
        # 默认都不包含
        keep = [False] * len(dets_sig_cls)
        for min_i in index[i+1:]:
            isin = find_inner_box_isin_outer_box(dets_sig_cls[max_i, 2:], dets_sig_cls[min_i, 2:])
            keep[min_i] = isin # 包含
        keeps.append(keep)
    # 取反,原本False为不包含,True为包含,取反后False为不保留,True为保留
    keeps = ~np.array(keeps)
    # print(keeps) # 每一行代表每个框相对于其他框是否要保留
    # [[True, True, True, True, False, True,  True,  True, True,  True,  True,  False],
    #  [True, True, True, True, True,  True,  True,  True, True,  True,  True,  True],
    #  [True, True, True, True, True,  True,  False, True, True,  True,  False, True],
    #  [True, True, True, True, True,  False, True,  True, False, False, True,  True],
    #  [True, True, True, True, True,  True,  True,  True, True,  True,  True,  True],
    #  [True, True, True, True, True,  True,  True,  True, True,  True,  False, True],
    #  [True, True, True, True, True,  True,  True,  True, True,  True,  True,  True],
    #  [True, True, True, True, True,  True,  True,  True, True,  True,  True,  True],
    #  [True, True, True, True, True,  True,  True,  True, True,  True,  True,  True],
    #  [True, True, True, True, True,  True,  True,  True, True,  True,  True,  True],
    #  [True, True, True, True, True,  True,  True,  True, True,  True,  True,  True]]

    # 最终保留的index,True/False
    # keeps.T: 转置之后每行代表是否要保留这个框
    final_keep = np.all(keeps.T, axis=-1)
    new_detections.append(dets_sig_cls[final_keep])

In [24]:
len(new_detections)

3

In [25]:
new_detections1 = np.concatenate(new_detections, axis=0)
new_detections1

array([[1.0000000e+01, 8.0000000e-01, 2.0004971e+02, 1.9626697e+02,
        4.8998325e+02, 4.2407892e+02],
       [1.0000000e+01, 5.0000000e-01, 1.7642613e+02, 0.0000000e+00,
        4.6068604e+02, 2.2706232e+02],
       [1.0000000e+01, 6.0000000e-01, 0.0000000e+00, 3.0343870e+02,
        1.7552124e+02, 4.2490558e+02],
       [1.0000000e+01, 7.0000000e-01, 1.4104881e+02, 3.1134420e+02,
        2.2894856e+02, 4.0853790e+02],
       [1.0000000e+01, 7.0000000e-01, 5.7525476e+02, 1.9562448e+02,
        6.2817926e+02, 2.9127210e+02],
       [1.1000000e+01, 6.0000000e-01, 4.5049182e+02, 1.8310547e+00,
        6.4000000e+02, 2.9299066e+02],
       [1.1000000e+01, 7.0000000e-01, 7.3793960e+01, 3.6816260e+02,
        7.9102310e+01, 3.7240448e+02],
       [1.2000000e+01, 8.0000000e-01, 5.9624290e+02, 2.4821837e+02,
        6.0194280e+02, 2.5399461e+02]])

In [27]:
h = new_detections1[:, 5] - new_detections1[:, 3]
w = new_detections1[:, 4] - new_detections1[:, 2]
h * w

array([6.60503251e+04, 6.45447146e+04, 2.13200174e+04, 8.54330193e+03,
       5.06210246e+03, 5.51771269e+04, 2.25173837e+01, 3.29239904e+01])